In [221]:
# All the libreries

#SQL
import mysql.connector
from mysql.connector import connect
from getpass import getpass

#Datetime
from datetime import datetime

#pandas
import pandas as pd
import numpy as np

#plots
import seaborn as sns
import matplotlib.pyplot as plt

#encode
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

#evaluation
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report


In [2]:
import mysql.connector

In [3]:
from mysql.connector import connect
from getpass import getpass

In [4]:
db = connect(
    host = 'localhost',
    user = 'root',
    passwd = getpass('Enter password: '),
    database = 'sakila'
)

Enter password: ········


 ## 1. Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features.

In [5]:
# cursor = instance used to execute SQL STATEMENTS on python
cursor = db.cursor()

In [6]:
#defining the query
query = 'select f.film_id, f.rental_duration, f.length, f.rating, f.replacement_cost, convert(r.rental_date,date) from film f left join inventory i using(film_id) left join rental r using(inventory_id)'
cursor.execute(query)

In [7]:
records = cursor.fetchall()

In [8]:
list = [record for record in records]


    #Another method to do it
    # for record in records:  
        #print(record)

In [9]:
list

[(1, 6, 86, 'PG', Decimal('20.99'), datetime.date(2005, 7, 8)),
 (1, 6, 86, 'PG', Decimal('20.99'), datetime.date(2005, 8, 2)),
 (1, 6, 86, 'PG', Decimal('20.99'), datetime.date(2005, 8, 21)),
 (1, 6, 86, 'PG', Decimal('20.99'), datetime.date(2005, 5, 30)),
 (1, 6, 86, 'PG', Decimal('20.99'), datetime.date(2005, 6, 17)),
 (1, 6, 86, 'PG', Decimal('20.99'), datetime.date(2005, 7, 7)),
 (1, 6, 86, 'PG', Decimal('20.99'), datetime.date(2005, 7, 30)),
 (1, 6, 86, 'PG', Decimal('20.99'), datetime.date(2005, 8, 23)),
 (1, 6, 86, 'PG', Decimal('20.99'), datetime.date(2005, 7, 31)),
 (1, 6, 86, 'PG', Decimal('20.99'), datetime.date(2005, 8, 22)),
 (1, 6, 86, 'PG', Decimal('20.99'), datetime.date(2005, 8, 2)),
 (1, 6, 86, 'PG', Decimal('20.99'), datetime.date(2005, 8, 21)),
 (1, 6, 86, 'PG', Decimal('20.99'), None),
 (1, 6, 86, 'PG', Decimal('20.99'), datetime.date(2005, 5, 27)),
 (1, 6, 86, 'PG', Decimal('20.99'), datetime.date(2005, 6, 21)),
 (1, 6, 86, 'PG', Decimal('20.99'), datetime.date(2

## 2. Read the data into a Pandas dataframe.

In [10]:
import pandas as pd

In [100]:
data = pd.DataFrame(records)

In [101]:
data.head()

,0,1,2,3,4,5
0,1,6,86,PG,20.99,2005-07-08
1,1,6,86,PG,20.99,2005-08-02
2,1,6,86,PG,20.99,2005-08-21
3,1,6,86,PG,20.99,2005-05-30
4,1,6,86,PG,20.99,2005-06-17


## 3. Analyze extracted features and transform them. You may need to encode some categorical variables, or scale numerical variables.

In [102]:
#changing the name of the columns
data.columns = ['film_id', 'rental_duration', 'length', 'rating', 'replacement_cost', 'rental_date']

In [103]:
data

,film_id,rental_duration,length,rating,replacement_cost,rental_date
0,1,6,86,PG,20.99,2005-07-08
1,1,6,86,PG,20.99,2005-08-02
2,1,6,86,PG,20.99,2005-08-21
3,1,6,86,PG,20.99,2005-05-30
4,1,6,86,PG,20.99,2005-06-17
...,...,...,...,...,...,...
16083,1000,3,50,NC-17,18.99,2005-05-29
16084,1000,3,50,NC-17,18.99,2005-06-15
16085,1000,3,50,NC-17,18.99,2005-07-12
16086,1000,3,50,NC-17,18.99,2005-07-31


In [15]:
#Elimination od duplicated rows

#print('Number of rows ' + str(data.shape[0]))

#print('Number of duplicated rows ' + str(data.duplicated().value_counts()[1]))

#Remove duplicated in the DataFrame
#data.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)

#print('Number of rows after removing duplicated rows ' + str(data.shape[0]))

Number of rows 16088
Number of duplicated rows 3426
Number of rows after removing duplicated rows 12662


In [16]:
data

,film_id,rental_duration,length,rating,replacement_cost,rental_date
0,1,6,86,PG,20.99,2005-07-08
1,1,6,86,PG,20.99,2005-08-02
2,1,6,86,PG,20.99,2005-08-21
3,1,6,86,PG,20.99,2005-05-30
4,1,6,86,PG,20.99,2005-06-17
...,...,...,...,...,...,...
16078,1000,3,50,NC-17,18.99,2005-07-09
16081,1000,3,50,NC-17,18.99,2005-08-01
16082,1000,3,50,NC-17,18.99,2005-08-23
16083,1000,3,50,NC-17,18.99,2005-05-29


In [138]:
#checking for Nan values
data.isna().sum()

film_id             0
rental_duration     0
length              0
rating              0
replacement_cost    0
rental_date         0
dtype: int64

In [139]:
#removing nan values
data = data.dropna(axis=0)
data.isna().sum()

film_id             0
rental_duration     0
length              0
rating              0
replacement_cost    0
rental_date         0
dtype: int64

In [140]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16045 entries, 0 to 16087
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   film_id           16045 non-null  int64  
 1   rental_duration   16045 non-null  int64  
 2   length            16045 non-null  int64  
 3   rating            16045 non-null  object 
 4   replacement_cost  16045 non-null  float64
 5   rental_date       16045 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 877.5+ KB


In [141]:
# Changing the type of the column 'replacement_cost' to float
data['replacement_cost'] = data['replacement_cost'].astype(float)

In [142]:
data.corr()

,film_id,rental_duration,length,replacement_cost
film_id,1.000000,-0.054767,0.056165,-0.038881
rental_duration,-0.054767,1.000000,0.055623,-0.026168
length,0.056165,0.055623,1.000000,0.026842
replacement_cost,-0.038881,-0.026168,0.026842,1.000000


In [143]:
data.describe(include='object')

,rating,rental_date
count,16045,16045
unique,5,42
top,PG-13,2005-07-31
freq,3585,679


In [144]:
data.describe(exclude='object')

,film_id,rental_duration,length,replacement_cost
count,16045.000000,16045.000000,16045.000000,16045.000000
mean,501.077719,4.935556,114.969274,20.215491
std,288.531551,1.401671,40.101750,6.081585
min,1.000000,3.000000,46.000000,9.990000
25%,255.000000,4.000000,81.000000,14.990000
50%,496.000000,5.000000,114.000000,20.990000
75%,753.000000,6.000000,148.000000,25.990000
max,1000.000000,7.000000,185.000000,29.990000


In [145]:
#Splitting the data into categorical and numerical

categorical = data.select_dtypes(include = object)
numerical = data.select_dtypes(exclude = object) 

In [146]:
numerical

,film_id,rental_duration,length,replacement_cost
0,1,6,86,20.99
1,1,6,86,20.99
2,1,6,86,20.99
3,1,6,86,20.99
4,1,6,86,20.99
...,...,...,...,...
16083,1000,3,50,18.99
16084,1000,3,50,18.99
16085,1000,3,50,18.99
16086,1000,3,50,18.99


#### Creating X

In [147]:
X = numerical

In [148]:
X

,film_id,rental_duration,length,replacement_cost
0,1,6,86,20.99
1,1,6,86,20.99
2,1,6,86,20.99
3,1,6,86,20.99
4,1,6,86,20.99
...,...,...,...,...
16083,1000,3,50,18.99
16084,1000,3,50,18.99
16085,1000,3,50,18.99
16086,1000,3,50,18.99


## 4. Create a query to get the list of films and a boolean indicating if it was rented last month. This would be our target variable.

In [149]:
#Creating the query

query1 = 'select f.film_id, f.title, cast(rental_date as date) from film f left join inventory i using(film_id) left join rental r using(inventory_id)'
cursor.execute(query1)
records1 = cursor.fetchall()

for record in records1:
    print(record)

(1, 'ACADEMY DINOSAUR', datetime.date(2005, 7, 8))
(1, 'ACADEMY DINOSAUR', datetime.date(2005, 8, 2))
(1, 'ACADEMY DINOSAUR', datetime.date(2005, 8, 21))
(1, 'ACADEMY DINOSAUR', datetime.date(2005, 5, 30))
(1, 'ACADEMY DINOSAUR', datetime.date(2005, 6, 17))
(1, 'ACADEMY DINOSAUR', datetime.date(2005, 7, 7))
(1, 'ACADEMY DINOSAUR', datetime.date(2005, 7, 30))
(1, 'ACADEMY DINOSAUR', datetime.date(2005, 8, 23))
(1, 'ACADEMY DINOSAUR', datetime.date(2005, 7, 31))
(1, 'ACADEMY DINOSAUR', datetime.date(2005, 8, 22))
(1, 'ACADEMY DINOSAUR', datetime.date(2005, 8, 2))
(1, 'ACADEMY DINOSAUR', datetime.date(2005, 8, 21))
(1, 'ACADEMY DINOSAUR', None)
(1, 'ACADEMY DINOSAUR', datetime.date(2005, 5, 27))
(1, 'ACADEMY DINOSAUR', datetime.date(2005, 6, 21))
(1, 'ACADEMY DINOSAUR', datetime.date(2005, 7, 7))
(1, 'ACADEMY DINOSAUR', datetime.date(2005, 7, 27))
(1, 'ACADEMY DINOSAUR', datetime.date(2005, 8, 21))
(1, 'ACADEMY DINOSAUR', datetime.date(2005, 6, 15))
(1, 'ACADEMY DINOSAUR', datetime.date(2

In [150]:
#Creating the dataframe of all the films rented

data_target = pd.DataFrame(records1)
display(data_target)

,0,1,2
0,1,ACADEMY DINOSAUR,2005-07-08
1,1,ACADEMY DINOSAUR,2005-08-02
2,1,ACADEMY DINOSAUR,2005-08-21
3,1,ACADEMY DINOSAUR,2005-05-30
4,1,ACADEMY DINOSAUR,2005-06-17
...,...,...,...
16083,1000,ZORRO ARK,2005-05-29
16084,1000,ZORRO ARK,2005-06-15
16085,1000,ZORRO ARK,2005-07-12
16086,1000,ZORRO ARK,2005-07-31


These are all the films rented last month

In [151]:
#changing the name of the columns
data_target.columns = ['film_id', 'title', 'rental_date']

In [152]:
data_target.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16088 entries, 0 to 16087
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   film_id      16088 non-null  int64 
 1   title        16088 non-null  object
 2   rental_date  16045 non-null  object
dtypes: int64(1), object(2)
memory usage: 377.2+ KB


In [153]:
#Checking for NaN values
data_target.isna().sum()

film_id         0
title           0
rental_date    43
dtype: int64

In [154]:
#removing nan values
data_target = data_target.dropna(axis=0)
data_target.isna().sum()

film_id        0
title          0
rental_date    0
dtype: int64

In [155]:
# Changing the type of the column 'rental_date' to datetime format

data_target["rental_date"] = pd.to_datetime(data_target["rental_date"], format='%Y-%m-%d')

In [156]:
# Creating the new column based in the boolean

data_target['last_month']= (data_target["rental_date"] >= '2005-08-01') & (data_target["rental_date"] <= '2005-08-31')

In [157]:
data_target['last_month'].value_counts()

False    10359
True      5686
Name: last_month, dtype: int64

In [246]:
# This is our target variable
y = data_target['last_month']

## 5. Create a logistic regression model to predict this variable from the cleaned data.

In [247]:
#Getting dummies of 'y'

y = pd.get_dummies(y, drop_first = True)

y.value_counts()

True
0       10359
1        5686
dtype: int64

#### '0' = False                                  '1' = True

#### Train/Test Split

In [234]:
X

,film_id,rental_duration,length,replacement_cost
0,1,6,86,20.99
1,1,6,86,20.99
2,1,6,86,20.99
3,1,6,86,20.99
4,1,6,86,20.99
...,...,...,...,...
16083,1000,3,50,18.99
16084,1000,3,50,18.99
16085,1000,3,50,18.99
16086,1000,3,50,18.99


In [248]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)

In [249]:
# Encoding the X varible

preprocessor = StandardScaler()

X_train = preprocessor.fit_transform(X_train)

X_test = preprocessor.transform(X_test)


In [237]:
# Training the model

classifier = LogisticRegression()

classifier.fit(X_train, y_train)

C:\Users\alber\anaconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [257]:
print("Train Score : {}".format(classifier.score(X_train, y_train)))
print('Test Score : {}'.format(classifier.score(X_test, y_test)))

Train Score : 0.6448223666636987
Test Score : 0.6379310344827587


In [238]:
# Calculating the predictions on train/test

In [239]:
# Train set
pred_train = classifier.predict(X_train)

#Test set
pred_test = classifier.predict(X_test)

## 6. Evaluate the results.

In [256]:
print(classification_report(y_train,pred_train))
print(classification_report(y_test, pred_test))

              precision    recall  f1-score   support

           0       0.64      1.00      0.78      7220
           1       0.00      0.00      0.00      4011

    accuracy                           0.64     11231
   macro avg       0.32      0.50      0.39     11231
weighted avg       0.41      0.64      0.50     11231

              precision    recall  f1-score   support

           0       0.65      1.00      0.79      3139
           1       0.00      0.00      0.00      1675

    accuracy                           0.65      4814
   macro avg       0.33      0.50      0.39      4814
weighted avg       0.43      0.65      0.51      4814



C:\Users\alber\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\alber\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\alber\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\alber\anaconda3\lib\site-packages\sklearn\metr

This model works really bad, it's absolutly unable to predict the '1' of our target variable